In [ ]:
import requests
import re
import os
from os import listdir
from os.path import isfile, join
import time
import datetime
import urllib.parse
import json
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver

In [ ]:
MATCH_PATH  = '/home/dudumint/桌面/dota2/data/raw_match/'
REPLAY_PATH = '/home/dudumint/桌面/dota2/data/raw_replay/'
# get match page html
def getMatchHtml(matchID):
    TRYNUM = 10
    
    openDotaUrl = "https://www.opendota.com/matches/{}".format(matchID)
    options = webdriver.ChromeOptions()
    
    browser = webdriver.Chrome()
    tryNum = 0
    getHTML= 0
    while tryNum <= TRYNUM :
        browser.implicitly_wait(20)
        browser.get(openDotaUrl)
        try:
            myDynamicElement = browser.find_element_by_class_name('matchButtons')
            getHTML= 1
            html = browser.page_source
            browser.close()
            print("No Fresh: ", tryNum)
            return html
        except Exception as e:
            browser.refresh()
            tryNum = tryNum+1
            print("Fresh: ", tryNum)
            continue
    return getHTML

def getImgHtml(matchID):
    TRYNUM = 10
    openDotaUrl = "https://www.opendota.com/matches/{}".format(matchID)
    options = webdriver.ChromeOptions()
    
    browser = webdriver.Chrome()
    tryNum = 0
    getHTML= 0
    while tryNum <= TRYNUM :
        browser.implicitly_wait(20)
        browser.get(openDotaUrl)
        try:
            myDynamicElement = browser.find_element_by_class_name('image')
            getHTML= 1
            html = browser.page_source
            browser.close()
            print("No Fresh: ", tryNum)
            return html
        except Exception as e:
            browser.refresh()
            tryNum = tryNum+1
            print("Fresh: ", tryNum)
            continue
    return getHTML

def download_file(url, file_path):
    from requests import get
    reply = get(url, stream=True)
    with open(file_path, 'wb') as file:
        for chunk in reply.iter_content(chunk_size=1024): 
            if chunk:
                file.write(chunk)
                
def findMatchFileByDate(dateStr):
    for element in MATCH_FILESNAME:
        m = re.match(dateStr, element)
        if m:
            return element
    return False

In [ ]:
MATCH_FILESNAME = listdir(MATCH_PATH)
for i in range(1,2):
    deltaday = datetime.date.today() - datetime.timedelta(days=i)
    year = deltaday.strftime("%Y")
    month = deltaday.strftime("%m")
    day = deltaday.strftime("%d")
    
    dateStr = '^{}{}{}'.format(year, month, day)
    
    matchFileName = '{}{}'.format(MATCH_PATH, findMatchFileByDate(dateStr))
    df = pd.read_csv(matchFileName)
    
    tmpMatchIdList = df['match_id']
    i = 0
    for tmpMatchId in tmpMatchIdList:
        # check file
        REPLAY_FILESNAME = listdir(REPLAY_PATH)
        if str(tmpMatchId)+".dem.bz2" in REPLAY_FILESNAME:
            print("Found: "+str(tmpMatchId)+".dem.bz2")
            print("{}/{}".format(i,len(tmpMatchIdList)))
            print()
            i += 1
            continue
        if str(tmpMatchId)+".dem" in REPLAY_FILESNAME:
            print("Found: "+str(tmpMatchId)+".dem")
            print("{}/{}".format(i,len(tmpMatchIdList)))
            print()
            i += 1
            continue
        # get heros info
        temp2 = getImgHtml(tmpMatchId)
        soup = BeautifulSoup(temp2)
        
        href_all = soup.findAll("a",attrs={'href' : re.compile("http://replay")})
        rpUrl = []
        for a in href_all:
            rpUrl = a['href']
            
        if rpUrl == []:
            continue

        fileName = rpUrl.split('/')
        fileName = fileName[len(fileName)-1].split('_')[0]
        fileName = '{}{}.dem.bz2'.format(REPLAY_PATH, fileName)
        download_file(rpUrl, fileName)
        
        print("Download: " + fileName)
        print(rpUrl)
        print("{}/{}".format(i,len(tmpMatchIdList)))
        print()
        i += 1
        time.sleep(1)
    